In [1]:
import pandas as pd
import numpy as np
import os
from tsfresh import extract_features, select_features
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute, impute_dataframe_zero
from pprint import pprint

/home/giotto/.local/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


**Dataset configuration**

In [2]:
dataset = 'synergy-final-iter5'

dataset_path = '../../datasets/' + dataset + '/'
output_path = '../../datasets/' + dataset + '-1s-processed/'
features_output_path = '../../datasets/' + dataset + '-1s-features/'

In [3]:
devices_to_use = [
    '128.237.246.127',
    '128.237.248.186',
    '128.237.247.134',
    '128.237.234.0',
    '128.237.237.122',
    '128.237.239.234',
    '128.237.254.195', # this Mite is only in the Synergy kitchen dataset
    '128.237.227.76',
    '128.237.247.190',
    '128.237.250.218',
    'DialogIoT 591844595',
    'DialogIoT 591844599',
    'DialogIoT 591844765',
    'Matrix b827eb96f31a',
    'Matrix b827ebe6e0f8',
    'Matrix b827eb41f96f',
    'TI SensorTag 604',
    'TI SensorTag 690',
    'TI SensorTag 85',
    'xdk_1',
    'xdk_2',
    'xdk_3',
    'TI SensorTag 33',
    'TI SensorTag 535',
    'TI SensorTag 709'
]

columns_to_rename = {
    'ACCEL_sst_0_avg': 'accel_x',
    'ACCEL_sst_1_avg': 'accel_y',
    'ACCEL_sst_2_avg': 'accel_z',
    'MAGNETOMETER_sst_0_avg': 'mag_x',
    'MAGNETOMETER_sst_1_avg': 'mag_y',
    'MAGNETOMETER_sst_2_avg': 'mag_z',
    'HUMIDITY_sst_0_avg': 'humidity',
    'ILLUMINATION_sst_0_avg': 'light',
    'BAROMETER_sst_0_avg': 'pressure',
    'TEMPERATURE_sst_0_avg': 'temperature',
    'MICROPHONE_sst_0_avg': 'microphone',
    'MICROPHONE_sst_0_min': 'microphone_min',
    'MICROPHONE_sst_0_max': 'microphone_max',
    'MICROPHONE_sst_0_sum': 'microphone_sum',
    'MICROPHONE_sst_0_variance': 'microphone_variance',
    'MICROPHONE_sst_0_range': 'microphone_range',
    'MICROPHONE_sst_0_centroid': 'microphone_centroid',
    'microphone_avg': 'microphone',
    'magnetometer_x': 'mag_x',
    'magnetometer_y': 'mag_y',
    'magnetometer_z': 'mag_z'
}

columns_to_keep = '|'.join([
    'accel_',
    'gyro_',
    'mag_',
    'humidity',
    'light',
    'pressure',
    'temperature',
    'microphone',
    'ACCEL_fft_',
    'ACCEL_sst_',
    'MICROPHONE_fft_',
    'EMI',
    'IRMOTION'
])

**Merge the individual CSVs into one DataFrame per device for the chosen devices**

In [5]:
data_frames = {}

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        device_id = None

        for device in devices_to_use:
            if file.startswith(device):
                device_id = device

        if device_id is not None:
            print(file)
            df = pd.DataFrame()
            if device_id in data_frames:
                df = data_frames[device_id]

            df_new = pd.DataFrame.from_csv(dataset_path + file)
            df_new.rename(index=str, columns=columns_to_rename, inplace=True)

            df_new.index = pd.to_datetime(df_new.index).round('100ms')

            df_new = df_new.filter(regex=(columns_to_keep))
#             df_new = df_new[df_new.columns.difference(['time'])]
            df_new = df_new[~df_new.index.duplicated(keep='first')]

            if len(df_new.columns) > 0:
                df = pd.concat([df, df_new], join='outer', axis=1)

                data_frames[device_id] = df

128.237.227.76_MICROPHONE_fft.csv


/home/giotto/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2787: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


128.237.250.218_HUMIDITY_sst.csv


/home/giotto/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2787: DtypeWarning: Columns (1,2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


128.237.227.76_BAROMETER_sst.csv
Matrix b827eb96f31a_alt_pre_tem_tim.csv
128.237.250.218_GEYE_sst.csv


...

128.237.250.218_BAROMETER_sst.csv
128.237.250.218_MAGNETOMETER_sst.csv


/home/giotto/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2787: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


128.237.250.218_ILLUMINATION_sst.csv
xdk_1_acc_gyr_hum_lig_mag_mic_pre_tem.csv
128.237.247.190_MICROPHONE_sst.csv
128.237.227.76_WIFI_RSSI_sst.csv
128.237.247.190_BAROMETER_sst.csv
128.237.227.76_GEYE_sst.csv
Matrix b827eb96f31a_mic_tim.csv
128.237.247.190_ILLUMINATION_sst.csv
128.237.227.76_ACCEL_sst.csv
128.237.247.190_WIFI_RSSI_sst.csv
128.237.227.76_TEMPERATURE_sst.csv
128.237.247.190_MACADDR_sst.csv
Matrix b827eb96f31a_acc_gyr_mag_pit_rol_tim_yaw.csv
128.237.250.218_ACCEL_fft.csv


/home/giotto/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2787: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,25

128.237.227.76_MACADDR_sst.csv
128.237.250.218_ACCEL_sst.csv
128.237.250.218_COLOR_sst.csv
128.237.247.190_EMI_sst.csv
Matrix b827eb41f96f_alt_pre_tem_tim.csv
Matrix b827ebe6e0f8_alt_pre_tem_tim.csv
128.237.250.218_MACADDR_sst.csv
128.237.247.190_TEMPERATURE_sst.csv
128.237.247.190_IRMOTION_sst.csv
Matrix b827eb41f96f_acc_gyr_mag_pit_rol_tim_yaw.csv
128.237.250.218_WIFI_RSSI_sst.csv
128.237.247.190_GEYE_sst.csv
Matrix b827ebe6e0f8_mic_tim.csv
128.237.227.76_ACCEL_fft.csv
128.237.247.190_HUMIDITY_sst.csv
128.237.227.76_COLOR_sst.csv
xdk_2_acc_gyr_hum_lig_mag_mic_pre_tem.csv
128.237.247.190_MICROPHONE_fft.csv
128.237.227.76_MAGNETOMETER_sst.csv
128.237.250.218_TEMPERATURE_sst.csv
128.237.227.76_IRMOTION_sst.csv
128.237.250.218_MICROPHONE_sst.csv
xdk_3_acc_gyr_hum_lig_mag_mic_pre_tem.csv
128.237.247.190_ACCEL_fft.csv
Matrix b827eb41f96f_mic_tim.csv
128.237.250.218_IRMOTION_sst.csv
128.237.227.76_ILLUMINATION_sst.csv
128.237.250.218_MICROPHONE_fft.csv
128.237.250.218_EMI_sst.csv
128.237.24

**Remove duplicate indices**

In [6]:
for device_id in data_frames:
    df = data_frames[device_id]
    df = df[~df.index.duplicated(keep='first')]
    data_frames[device_id] = df

**Convert index to datetime**

In [7]:
for device_id in data_frames:
    df = data_frames[device_id]
    df.index = pd.to_datetime(df.index)
    data_frames[device_id] = df

**Fill missing values**

In [8]:
for device_id in data_frames:
    df = data_frames[device_id]
    df = df.ffill().bfill()
    data_frames[device_id] = df

**Create 1 sec activity windows**

In [ ]:
window_size_secs = 1 # seconds
smallest_size_secs = 1 # seconds

activities = pd.DataFrame.from_csv(dataset_path + 'activities.csv')
labels = pd.DataFrame.from_csv(dataset_path + 'activity_labels.csv')
labeled_activities = activities.loc[activities.id != -1]

activity_windows = []

for id in labeled_activities.id.unique():
    start = labeled_activities.loc[labeled_activities.id == id].index.min()
    end = labeled_activities.loc[labeled_activities.id == id].index.max()

    since = start
    until = start + pd.DateOffset(seconds=window_size_secs)

    while since < end:
        label = labels.loc[id]['label']

        length = (until - since).seconds
        if length >= smallest_size_secs:
            activity_windows.append({
                'since': since,
                'until': until,
                'label': label,
                'length': length,
                'id': id
            })

        since = until
        until = until + pd.DateOffset(seconds=window_size_secs)
        until = min(until, end)

**Create per-activity dataframes which are not missing more than 1 second of data**

In [ ]:
activity_data_frames = {}
device_labels = {}

removed_activities = {}
all_activities = {}
activity_id = 0

for device_id in data_frames:
    df = data_frames[device_id]
    data_frames[device_id] = None
    df['id'] = -1

    device_activities = []

    for i, window in enumerate(activity_windows):

        activity_df = df.loc[df.index >= window['since']]
        activity_df = activity_df.loc[activity_df.index <= window['until']].copy()

        seconds = activity_df.index.round('s').unique()

        key = device_id + ', activity ' + str(window['label'])
        if not key in all_activities:
            all_activities[key] = 0
            removed_activities[key] = 0

        if len(seconds) < window['length'] or len(activity_df) <= 3:
            removed_activities[key] += 1
        else:
            activity_df['id'] = activity_id
            activity_data_frames[activity_id] = activity_df
            device_activities.append(pd.DataFrame({
                'id': [activity_id],
                'label': [window['label']],
                'activity_id': [window['id']],
                'since': [window['since']],
                'until': [window['until']],
                'window_id': [i]
            }))
            activity_id += 1

        all_activities[key] += 1

    df_labels = pd.concat(device_activities)
    df_labels = df_labels.set_index('id')
    device_labels[device_id] = df_labels

# print stats    
for key in all_activities:
    if removed_activities[key] > 0:
        print(str(removed_activities[key]) + ' out of ' + str(all_activities[key]) + ' removed for ' + key)

4 out of 31 removed for 128.237.247.190, activity 24
221 out of 393 removed for 128.237.250.218, activity 1
10 out of 31 removed for Matrix b827eb96f31a, activity 24
45 out of 384 removed for Matrix b827eb96f31a, activity 23
49 out of 393 removed for 128.237.227.76, activity 1
29 out of 393 removed for xdk_1, activity 1
16 out of 393 removed for Matrix b827eb41f96f, activity 1
60 out of 393 removed for xdk_3, activity 1
1 out of 348 removed for xdk_1, activity 21
92 out of 377 removed for Matrix b827eb96f31a, activity 22
43 out of 384 removed for Matrix b827eb96f31a, activity 2
2 out of 376 removed for 128.237.247.190, activity 0
36 out of 393 removed for Matrix b827ebe6e0f8, activity 1
1 out of 381 removed for 128.237.247.190, activity 3
1 out of 368 removed for 128.237.247.190, activity 7
151 out of 348 removed for Matrix b827eb96f31a, activity 21
15 out of 381 removed for Matrix b827eb96f31a, activity 3
33 out of 376 removed for Matrix b827eb96f31a, activity 0
25 out of 368 removed 

**Resample to 10Hz and fill missing values**

In [ ]:
for activity_id in activity_data_frames:
    df = activity_data_frames[activity_id]
    df = df.resample('100L')
    df = df.ffill().bfill()
    activity_data_frames[activity_id] = df

**Update device data frames**

In [ ]:
for device_id in device_labels:
    labels = device_labels[device_id]
    df_parts = [activity_data_frames[id] for id in labels.index]
    df = pd.concat(df_parts)
    data_frames[device_id] = df

**Replace timestamp indices with integer ones**

In [ ]:
for device_id in data_frames:
    df = data_frames[device_id]
    df.insert(0, 'i_index', pd.Series(range(len(df.index)), index=df.index))
    df = df.set_index(['i_index'])
    data_frames[device_id] = df

**Save as Pickle files to disk**

In [ ]:
for device_id in data_frames:
    df = data_frames[device_id]
    df.to_pickle(output_path + device_id + '.p')

    df_labels = device_labels[device_id]
    df_labels.to_pickle(output_path + device_id + '_labels.p')

**Reload extracted datasets from disk**

In [4]:
data_frames = {}
device_labels = {}

for root, dirs, files in os.walk(output_path):
    for file in files:
        device_id = None

        for device in devices_to_use:
            if file.startswith(device):
                device_id = device

        if device_id is not None:
            df = pd.read_pickle(output_path + file)
            if file.endswith('_labels.p'):
                device_labels[device_id] = df
            else:
                data_frames[device_id] = df

**Extract features**

In [ ]:
data_frame_features = {}

def extract_device_features(df):
    df = df.astype(float)
    impute(df)

    fft_columns = [col for col in df.columns if '_fft_' in col]
    non_fft_columns = [col for col in df.columns if '_fft_' not in col]

    if len(fft_columns) > 0:
        fft_columns = fft_columns + ['id']

        df_fft = df[fft_columns]
        df_non_fft = df[non_fft_columns]

        print('Start extracting FFT features')
        extracted_features_fft = extract_features(df_fft,
                                              column_id="id",
                                              default_fc_parameters=MinimalFCParameters())

        print('Start extracting non-FFT features')
        extracted_features_non_fft = extract_features(df_non_fft,
                                              column_id="id",
                                              default_fc_parameters=EfficientFCParameters())

        extracted_features = pd.concat([extracted_features_fft, extracted_features_non_fft], axis=1)
    else:
        print('Start extracting non-FFT features')
        extracted_features = extract_features(df,
                                              column_id="id",
                                              default_fc_parameters=EfficientFCParameters())
    print('Finished extracting features')
    impute_dataframe_zero(extracted_features)
    print('Imputed data')

    return extracted_features

for device_id in data_frames:
    print(device_id)
    df = data_frames[device_id]

    extracted = extract_device_features(df)
    data_frame_features[device_id] = extracted

128.237.247.190
Start extracting FFT features


Feature Extraction: 100%|██████████| 512/512 [02:52<00:00,  2.96it/s] 


Start extracting non-FFT features


Feature Extraction: 100%|██████████| 49/49 [19:28<00:00, 16.68s/it]  


Finished extracting features
Imputed data
Matrix b827eb96f31a
Start extracting non-FFT features


...

Finished extracting features
Imputed data
xdk_3
Start extracting non-FFT features


Feature Extraction: 100%|██████████| 14/14 [06:30<00:00, 29.11s/it]


Finished extracting features
Imputed data
xdk_1
Start extracting non-FFT features


Feature Extraction: 100%|██████████| 14/14 [06:29<00:00, 27.85s/it]


Finished extracting features
Imputed data


**Sort feature columns**

In [ ]:
for device_id in data_frame_features:
    df = data_frame_features[device_id]
    df = df.reindex_axis(sorted(df.columns), axis=1)
    data_frame_features[device_id] = df

**Store extracted features on disk**

In [ ]:
for device_id in data_frame_features:
    extracted = data_frame_features[device_id]
    extracted.to_pickle(features_output_path + device_id + '.p')

    df_labels = device_labels[device_id]
    df_labels.to_pickle(features_output_path + device_id + '_labels.p')

**Do feature selection**

In [ ]:
data_frame_selected = {}

for device_id in device_labels:
    df = data_frame_features[device_id]
    df_labels = device_labels[device_id]
    selected = select_features(df, df_labels['label'])
    data_frame_selected[device_id] = selected

**Save to disk**

In [ ]:
for device_id in data_frame_selected:
    selected = data_frame_selected[device_id]
    selected.to_pickle(features_output_path + device_id + '_selected.p')

In [ ]:
1

In [ ]:
1